In [1]:
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
def cal_delay_spread(delay):
    # data is a list, may include inf
    temp = delay[:]
    temp.sort()
    length = len(temp)
    return temp[round(length*0.75)] - temp[round(length*0.25)]

In [3]:
# process nameserver data
def ns_data():
    time = [[], [], []]
    ping = [[], [], []]
    dig_time = [[], [], []]
    f = open('nsdata.txt')
    for line in f:
        if 'name server' in line:
            curser = line.split(' ')[2].split(':')[0]
            curser = int(curser)
            continue
        if 'Query time' in line:
            qtime = int(line.split(' ')[3])
            dig_time[curser-1].append(qtime)
            continue    
        if '64 bytes' in line:
            date = float(line.split('[')[1].split(']')[0])
            rtt = line.split('time=')[1].split('ms')[0]
            time[curser-1].append(date)
            ping[curser-1].append(float(rtt))
            continue
        if 'no answer' in line:
            date = float(line.split('[')[1].split(']')[0])
            time[curser-1].append(date)
            ping[curser-1].append(float('inf'))
            continue
        
    # packet_loss
    p_loss = []
    for ping_i in ping:
        t = ping_i.count(float('inf'))
        p_loss.append(t)
    # median delay(loss counted)
    med_ping = [np.median(ping[0]),np.median(ping[1]),np.median(ping[2])]
    med_dig = [np.median(dig_time[0]),np.median(dig_time[1]),np.median(dig_time[2])]
    # loss ratio
    lratio = [p_loss[0]/len(ping[0]), p_loss[1]/len(ping[1]), p_loss[2]/len(ping[2])]
    # delay spread
    ds_ping= [cal_delay_spread(ping[0]), cal_delay_spread(ping[1]), cal_delay_spread(ping[2])]
    ds_dig = [cal_delay_spread(dig_time[0]), cal_delay_spread(dig_time[1]), cal_delay_spread(dig_time[2])]
    # mean delay(loss not counted)
    ping[0][:] = [x for x in ping[0] if x != float('inf')]
    ping[1][:] = [x for x in ping[1] if x != float('inf')]
    ping[2][:] = [x for x in ping[2] if x != float('inf')]
    mean_ping = [np.mean(ping[0]),np.mean(ping[1]),np.mean(ping[2])]
    mean_dig = [np.mean(dig_time[0]),np.mean(dig_time[1]),np.mean(dig_time[2])]
    ns_ping = [mean_ping, med_ping, lratio, ds_ping]
    ns_dig = [mean_dig, med_dig, lratio, ds_dig]
    
    return ns_ping, ns_dig

ns_ping, ns_dig = ns_data()

In [4]:
# process research server data
def rs_data():
    f = open('rsdata.txt')
    time = [[],[],[]]
    ping = [[],[],[]]
    
    for line in f:
        if 'research server' in line:
            curser = int(line.split(' ')[2].split(':')[0])
            continue
        if '64 bytes' in line:
            date = float(line.split('[')[1].split(']')[0])
            rtt = float(line.split('time=')[1].split(' ')[0])
            time[curser-1].append(date)
            ping[curser-1].append(rtt)
            continue
        if 'no answer' in line:
            date = float(line.split('[')[1].split(']')[0])
            rtt = float('inf')
            time[curser-1].append(date)
            ping[curser-1].append(rtt)

    # packet_loss
    p_loss = []
    for ping_i in ping:
        t = ping_i.count(float('inf'))
        p_loss.append(t)
    
    # median delay
    med_ping = [np.median(ping[0]),np.median(ping[1]),np.median(ping[2])]
    # loss ratio
    lratio = [p_loss[0]/len(ping[0]), p_loss[1]/len(ping[1]), p_loss[2]/len(ping[2])]
    # delay spread
    ds_ping = [cal_delay_spread(ping[0]), cal_delay_spread(ping[1]), cal_delay_spread(ping[2])]
    # mean delay
    ping[0][:] = [x for x in ping[0] if x != float('inf')]
    ping[1][:] = [x for x in ping[1] if x != float('inf')]
    ping[2][:] = [x for x in ping[2] if x != float('inf')]
    mean_ping = [np.mean(ping[0]), np.mean(ping[1]), np.mean(ping[2])]
    rs_ping = [med_ping, mean_ping, lratio, ds_ping]
    return rs_ping
        
rs_ping = rs_data()

In [5]:
# process iperf server data
def is_data():
    f = open('isdata.txt')
    time = [[], []]
    ping = [[], []]
    curl = [[], []]  # store the value of connect-namelookup
    temp = 0
    
    for line in f:
        if 'time_namelookup' in line:
            temp = 1
            continue
        if temp == 1:
            namelookup = float(line.split(', ')[0])
            connect = float(line.split(', ')[1])
            tcp_latency = (connect - namelookup)*1000
            tcp_latency = '{:.6f}'.format(tcp_latency)
            curl[curser-1].append(float(tcp_latency))
            temp = 0
            continue
        if 'iperf server' in line:
            curser = int(line.split(' ')[2].split(':')[0])
            continue
        if '64 bytes' in line:
            date = float(line.split('[')[1].split(']')[0])
            rtt = float(line.split('time=')[1].split(' ')[0])
            time[curser-1].append(date)
            ping[curser-1].append(rtt)
            continue
        if 'no answer' in line:
            date = float(line.split('[')[1].split(']')[0])
            rtt = float('inf')
            time[curser-1].append(date)
            ping[curser-1].append(rtt)

    # packet_loss
    p_loss = []
    for ping_i in ping:
        t = ping_i.count(float('inf'))
        p_loss.append(t)
    # median delay(loss counted)
    med_ping = [np.median(ping[0]), np.median(ping[1])]
    med_curl = [np.median(curl[0]), np.median(curl[1])]
    # loss ratio
    lratio = [p_loss[0]/len(ping[0]), p_loss[1]/len(ping[1])]
    # delay spread
    ds_ping = [cal_delay_spread(ping[0]), cal_delay_spread(ping[1])]
    ds_curl = [cal_delay_spread(curl[0]), cal_delay_spread(curl[1])]
    # mean delay
    mean_ping = [np.mean(ping[0]),np.mean(ping[1])]
    mean_curl = [np.mean(curl[0]),np.mean(curl[1])]

    is_ping = [med_ping, mean_ping, lratio, ds_ping]
    is_curl = [med_curl, mean_curl, lratio, ds_curl]

    return is_ping, is_curl
    

is_ping, is_curl = is_data()

In [6]:
from IPython.display import display

def disp_table():
    server_name = {
        'Server':['d.nic.fr',
        'e.ext.nic.fr',
        'f.ext.nic.fr',
        'lej-de.ark.caida.org',
        'per-au.ark.caida.org',
        'san-us.ark.caida.org',
        'ok1.iperf.comnet-student.eu',
        'blr1.iperf.comnet-student.eu'],
        'Name':['name 1', 
        'name 2', 
        'name 3', 
        'research 1', 
        'research 2', 
        'research 3',
        'iperf 1',
        'iperf 2']
    }
    df = pd.DataFrame(server_name)
    display(df)

    item=['name 1 ICMP',
        'name 2 ICMP',
        'name 3 ICMP',
        'name 1 DNS query',
        'name 2 DNS query',
        'name 3 DNS query',
        'research 1 ICMP',
        'research 2 ICMP',
        'research 3 ICMP',
        'iperf 1 ICMP',
        'iperf 2 ICMP',
        'iperf 1 HTTP',
        'iperf 2 HTTP']
    table_data = {
        'Server&method':item,
        'Median delay(ms)': ns_ping[0]+ ns_dig[0]+ rs_ping[0]+ is_ping[0]+ is_curl[0] ,
        'Mean delay(ms)': ns_ping[1]+ ns_dig[1]+ rs_ping[1]+ is_ping[1]+ is_curl[1] ,
        'Loss ratio': ns_ping[2]+ ns_dig[2]+ rs_ping[2]+ is_ping[2]+ is_curl[2] ,
        'Delay spread(ms)': ns_ping[3]+ ns_dig[3]+ rs_ping[3]+ is_ping[3]+ is_curl[3] ,
    }
    df = pd.DataFrame(table_data)
    display(df)

disp_table()


,Server,Name
0,d.nic.fr,name 1
1,e.ext.nic.fr,name 2
2,f.ext.nic.fr,name 3
3,lej-de.ark.caida.org,research 1
4,per-au.ark.caida.org,research 2
5,san-us.ark.caida.org,research 3
6,ok1.iperf.comnet-student.eu,iperf 1
7,blr1.iperf.comnet-student.eu,iperf 2


,Server&method,Median delay(ms),Mean delay(ms),Loss ratio,Delay spread(ms)
0,name 1 ICMP,91.737500,91.700000,0.0,0.100
1,name 2 ICMP,27.029167,26.850000,0.0,0.400
2,name 3 ICMP,0.990083,0.887000,0.0,0.388
3,name 1 DNS query,92.000000,92.000000,0.0,0.000
4,name 2 DNS query,27.000000,28.000000,0.0,0.000
5,name 3 DNS query,1.000000,0.000000,0.0,4.000
6,research 1 ICMP,25.800000,25.834861,0.0,0.000
7,research 2 ICMP,244.000000,244.084722,0.0,0.000
8,research 3 ICMP,175.000000,177.072222,0.0,0.000
9,iperf 1 ICMP,0.460500,1.177631,0.0,1.503
